### Song Name Maker

* Uses either Chord/Shahin
* Or Holoviews

In [1]:
# base
import pandas as pd
import numpy as np
import nltk
from nltk import bigrams
import itertools
import pandas as pd
import re

# Chord/Shahin
from chord import Chord

# hv
import holoviews as hv
from holoviews import opts, dim

### Load data

In [2]:
# reading csv file  
# matched = pd.read_csv(r"../Py code/words_songs_matched.csv") 

In [3]:
# matched.to_csv(r'../Py code/words_songs_matched.zip', index = False)

In [2]:
# reading csv file  
matched = pd.read_csv(r"../Py code/words_songs_matched.zip") 

In [3]:
# check the data
print(matched.shape)
matched.head()

(19045332, 10)


,word,count,track_id,song_id,artist_id,title,artist_name,duration,year,artist_hotttnesss
0,i,6,TRAAAAV128F421A322,SOQPWCR12A6D4FB2A3,AR73AIO1187B9AD57B,A Poor Recipe For Civic Cohesion,Western Addiction,118.07302,2005,0.386606
1,the,4,TRAAAAV128F421A322,SOQPWCR12A6D4FB2A3,AR73AIO1187B9AD57B,A Poor Recipe For Civic Cohesion,Western Addiction,118.07302,2005,0.386606
2,you,2,TRAAAAV128F421A322,SOQPWCR12A6D4FB2A3,AR73AIO1187B9AD57B,A Poor Recipe For Civic Cohesion,Western Addiction,118.07302,2005,0.386606
3,to,2,TRAAAAV128F421A322,SOQPWCR12A6D4FB2A3,AR73AIO1187B9AD57B,A Poor Recipe For Civic Cohesion,Western Addiction,118.07302,2005,0.386606
4,and,5,TRAAAAV128F421A322,SOQPWCR12A6D4FB2A3,AR73AIO1187B9AD57B,A Poor Recipe For Civic Cohesion,Western Addiction,118.07302,2005,0.386606


### Selective data drop

In [4]:
# no longer used, caution INPLACE

dropcols = ['track_id', 'song_id', 'artist_id', 'duration']

# matched.drop(columns=dropcol, inplace=True)
# matched.head()

### Utility Functions

In [5]:
# remove parentheses and contents from song titles (or any string input)

def Clean_titles(title): 
    # Search for opening bracket in the name followed by 
    # any characters repeated any number of times
    if re.search('\(.*', str(title)):
        # return the cleaned name 
        return re.sub(r'\([^)]*\)', '', str(title)) # FORCE TO STRING in case of floats
    else: 
        # if clean up needed return the same name 
        return title 

In [75]:
# build co-occurrence matrix
# input: list of lists
# output: outmatrix

def generate_co_occurrence_matrix(corpus):
    vocab = set(corpus)
    print('vocab len: ' + str(len(vocab)))
    vocab = list(vocab)
    vocab_index = {word: i for i, word in enumerate(vocab)}

    # Create bigrams from all words in corpus
    bi_grams = list(bigrams(corpus))

    # Frequency distribution of bigrams ((word1, word2), num_occurrences)
    bigram_freqraw = nltk.FreqDist(bi_grams).most_common(len(bi_grams))
    bigram_freq = [bigram for bigram in bigram_freqraw if bigram[1] > 1]

    # Initialise co-occurrence matrix
    # co_occurrence_matrix[current][previous]
    co_occurrence_matrix = np.zeros((len(vocab), len(vocab)))

    # Loop through the bigrams taking the current and previous word,
    # and the number of occurrences of the bigram.
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]
        pos_current = vocab_index[current]
        pos_previous = vocab_index[previous]
        co_occurrence_matrix[pos_current][pos_previous] = count
    co_occurrence_matrix = np.matrix(co_occurrence_matrix)

    # return the matrix and the index
    return co_occurrence_matrix, bigram_freq, vocab_index

In [179]:
# strip cooccurence matrix of low value weights

def rematrix(df, limit):
    for idx in df.index:
        rowsum = df.loc[idx,:].sum(axis=0)
        colsum = df[idx].sum(axis=0)
#         print('Idx: ' + idx + '; sum of row: ' + str(rowsum))
        if rowsum < limit and colsum < limit:
            df.drop([idx], axis = 0, inplace=True)
            df.drop([idx], axis = 1, inplace=True)
#             print('dropping: ' + idx)
    return df

In [178]:
# generate matrix for chord/shahin
# limit is min co-occur #

def gen_chord(inlist, limit):
    data = list(itertools.chain.from_iterable(inlist))
    matrix, bigram, vocab_index = generate_co_occurrence_matrix(data)

    data_matrix = pd.DataFrame(matrix, index=vocab_index, columns=vocab_index)
    
    data_matrix = rematrix(data_matrix, limit)
    
    outmatrix = data_matrix.to_numpy()
    outmatrix = outmatrix.tolist()
#     outmatrix = matrix.tolist()
    names = list(data_matrix.index) 

#     print(matrix)
#     print(type(matrix))
#     print(data_matrix)
#     print(type(names))
#     print(names)
    return outmatrix, data_matrix, bigram, names

In [184]:
# generate chord chart to html
# input: matrix, names, output name
# output: html file saved to same directory

def gen_html(matrix, names, output):
    Chord(matrix, names, padding=0.1, width=1200, wrap_labels=False).to_html(output)

In [9]:
# remove stopwords
# input: df, column name containing words
# output: df with stopwords removed

def rm_stopwords(df, name):
    nltk.download('stopwords')
    from nltk.corpus import stopwords
    sw = stopwords.words('english')
    realwords = df[~df[name].isin(sw)]
    return realwords

In [10]:
# create a matrix for word in each song
# input: df with column of words, one per row, multiple duplicates of song title
# df 'realwords'
# column 'title'

def longtable_to_listoflist(realwords):

    finallist= []
    wordlist = []

    uniq_title = realwords.title.unique()

    for song in uniq_title:
        wordlist = []
        print(song)
        subset = realwords[realwords['title']==song]
        for word in subset['word']:
            wordlist.append(word)
        finallist.append(wordlist)

    print(finallist)
    return finallist

### Extract unique songs, hotties, notties

In [11]:
# drop duplicates against song title
deduped = matched.drop_duplicates(subset="title", keep='first', inplace=False)
print(deduped.shape)
deduped.head()

(190525, 10)


,word,count,track_id,song_id,artist_id,title,artist_name,duration,year,artist_hotttnesss
0,i,6,TRAAAAV128F421A322,SOQPWCR12A6D4FB2A3,AR73AIO1187B9AD57B,A Poor Recipe For Civic Cohesion,Western Addiction,118.07302,2005,0.386606
68,i,10,TRAAABD128F429CF47,SOCIWDW12A8C13D406,ARMJAGH1187FB546F3,Soul Deep,The Box Tops,148.03546,1969,0.417500
142,i,28,TRAAAED128E0783FAB,SOXZYWX12A6310ED0C,ARC1IHZ1187FB4E920,It's About Time,Jamie Cullum,246.98730,0,0.562061
262,i,5,TRAAAEF128F4273421,SONHOTT12A8C13493C,AR7G5I41187FB4CE6C,Something Girls,Adam Ant,233.40363,1982,0.454231
321,i,4,TRAAAEW128F42930C0,SODZYPO12A8C13A91E,AR1C2IX1187B99BF74,Burn My Body (Album Version),Broken Spindles,177.99791,0,0.378545


In [12]:
deduped['title'] = deduped['title'].apply(Clean_titles)
print(deduped.shape)
deduped.head()

(190525, 10)


C:\Users\Wx\.conda\envs\dv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,word,count,track_id,song_id,artist_id,title,artist_name,duration,year,artist_hotttnesss
0,i,6,TRAAAAV128F421A322,SOQPWCR12A6D4FB2A3,AR73AIO1187B9AD57B,A Poor Recipe For Civic Cohesion,Western Addiction,118.07302,2005,0.386606
68,i,10,TRAAABD128F429CF47,SOCIWDW12A8C13D406,ARMJAGH1187FB546F3,Soul Deep,The Box Tops,148.03546,1969,0.417500
142,i,28,TRAAAED128E0783FAB,SOXZYWX12A6310ED0C,ARC1IHZ1187FB4E920,It's About Time,Jamie Cullum,246.98730,0,0.562061
262,i,5,TRAAAEF128F4273421,SONHOTT12A8C13493C,AR7G5I41187FB4CE6C,Something Girls,Adam Ant,233.40363,1982,0.454231
321,i,4,TRAAAEW128F42930C0,SODZYPO12A8C13A91E,AR1C2IX1187B99BF74,Burn My Body,Broken Spindles,177.99791,0,0.378545


## RESTART HERE

In [13]:
deduped.to_csv(r"../Py code/titles_clean.zip") 

In [14]:
deduped = pd.read_csv(r"../Py code/titles_clean.zip")
deduped.head()

,Unnamed: 0,word,count,track_id,song_id,artist_id,title,artist_name,duration,year,artist_hotttnesss
0,0,i,6,TRAAAAV128F421A322,SOQPWCR12A6D4FB2A3,AR73AIO1187B9AD57B,A Poor Recipe For Civic Cohesion,Western Addiction,118.07302,2005,0.386606
1,68,i,10,TRAAABD128F429CF47,SOCIWDW12A8C13D406,ARMJAGH1187FB546F3,Soul Deep,The Box Tops,148.03546,1969,0.417500
2,142,i,28,TRAAAED128E0783FAB,SOXZYWX12A6310ED0C,ARC1IHZ1187FB4E920,It's About Time,Jamie Cullum,246.98730,0,0.562061
3,262,i,5,TRAAAEF128F4273421,SONHOTT12A8C13493C,AR7G5I41187FB4CE6C,Something Girls,Adam Ant,233.40363,1982,0.454231
4,321,i,4,TRAAAEW128F42930C0,SODZYPO12A8C13A91E,AR1C2IX1187B99BF74,Burn My Body,Broken Spindles,177.99791,0,0.378545


In [105]:
# Set thresholds 

hot_thresh = 1
not_thresh = 0.2

In [106]:
# filter down to hotties
hotties = deduped[deduped['artist_hotttnesss'] > hot_thresh]
hotties.drop(columns=dropcols, inplace=True)
unique_hotties = hotties.title.unique()
print(str(len(unique_hotties)) + ' unique songs with hotness greater than ' + str(hot_thresh))
print(unique_hotties)

83 unique songs with hotness greater than 1
["Let's Get It Started" 'On My Own' "Heard 'Em Say" 'Skit #2 '
 'School Spirit Skit 2' 'Tell Your Momma Come' 'Boom Boom Guetta'
 'Touch It / Technologic' "Lil' Lil'" 'Anxiety' 'I Gotta Feeling '
 'Too Long' 'Harder Better Faster Stronger' 'Third Eye' 'Late'
 'All Falls Down' 'One More Time / Aerodynamic' 'One More Time '
 'Gone Going' 'Something About Us ' 'Human After All' 'Skit #1 ' 'Pump It'
 "Rock'n Roll" 'Head Bobs' 'Jesus Walks' 'Hey Mama' 'Wake Up Mr. West'
 'Everything I Am' 'Rockin To The Beat' 'The New Workout Plan'
 'Get Em High' 'Revolution 909' "Can't Tell Me Nothing" 'Street Lights'
 'Dum Diddly' 'Request + Line' "What's Goin Down" 'Superhereos' 'Skit #3 '
 'Big Brother' 'The Prime Time Of Your Life' 'One Tribe'
 'Do What You Want' 'RoboCop' 'If You Want Love' 'Disco Club'
 'Face To Face ' 'Too Long ' 'The Brainwasher' 'Get Original'
 'Television Rules The Nation' 'Something About Us' 'Aerodynamic'
 'Boom Boom Pow' 'My Humps' '

In [107]:
# filter down to NOTties, doing yourself no favors with naming convention here
notties = deduped[(deduped['artist_hotttnesss'] < not_thresh) & (deduped['artist_hotttnesss'] > 0)]
notties.drop(columns=dropcols, inplace=True)
unique_notties = notties.title.unique()
print(str(len(unique_notties)) + ' unique songs with hotness less than ' + str(not_thresh))
print(unique_notties)

553 unique songs with hotness less than 0.2
['Next Time' "Don't Cry" 'I\x19m Moving On' 'Seaside'
 'Just a Little Bit of Love' "I Don't Know" 'Scarred' 'Tania' "C'mon"
 'Is It You?' "At Least It's Raining" 'Boy! What Love Has to Me!'
 "It's Alright" 'Humain' 'Out Of Tune' 'Supersonic' 'Why You Lie'
 'Party Favours' 'boy for sale' 'Al Final del Sendero' 'Italian Song'
 "The Worst Holiday I've Ever Had" 'Fire Me Up' 'All He Ever Wanted'
 "Let's Make Love" 'We Woke Up A Fire ' 'Sjukbesök'
 'She Moved through the Fair' 'Credits' 'Josy' 'Never Happy'
 "Don't Do It Again" "Why Don't You Love Me" 'Big Red Machine'
 '7-Eleven Christmas' 'Child Of God' 'Atomic Dog' 'Se 101'
 'Tickle Yore Innards' 'Magick Christian' "I'm A Fool " 'Words Of Love'
 "Director's Cut" 'Talking Drum' 'Love and Hate' 'Buenas Tardes'
 'Les amis de tes amis' 'Rattletrap' 'Bop Gun' 'Paradise Passage Road'
 "Keep on movin'" 'Dirty Habits' 'Ficção Cientifica'
 "You're Driving Me Crazy" 'Famous Last Words' 'My Kitten'
 'Lais

### Build the title data

In [108]:
# create a matrix for word in each song
# input: list of song names
# output: list of lists, each song name split into component words

hotlist = []
notlist = []

for song in unique_hotties:
    subset = song.split()
    hotlist.append(subset)
    
for song in unique_notties:
    subset = song.split()
    notlist.append(subset)
        
print('Hotlist: ')
# print(hotlist)
print('\nNotlist: ')
# print(notlist)

Hotlist: 

Notlist: 


In [180]:
limit = 1
hot_mat, hot_data, hot_bigrams, hot_names = gen_chord(hotlist, limit)

vocab len: 156


In [116]:
hot_bigrams

# for bigram in hot_bigrams:
#     current = bigram[0][1]
#     print('current: ' + current)
#     previous = bigram[0][0]
#     print('prev: ' + previous)
#     count = bigram[1]
#     print('count: ' + str(count))

[(('Too', 'Long'), 4),
 (('School', 'Spirit'), 2),
 (('Spirit', 'Skit'), 2),
 (('Boom', 'Boom'), 2),
 (('Harder', 'Better'), 2),
 (('Better', 'Faster'), 2),
 (('Faster', 'Stronger'), 2),
 (('One', 'More'), 2),
 (('More', 'Time'), 2),
 (('Something', 'About'), 2),
 (('About', 'Us'), 2),
 (('Workout', 'Plan'), 2),
 (('You', 'Want'), 2),
 (('Television', 'Rules'), 2),
 (('Rules', 'The'), 2),
 (('The', 'Nation'), 2),
 (('Around', 'The'), 2),
 (('The', 'World'), 2)]

In [59]:
z = len(hot_bigrams)
for i in range(z):
    if hotbigrams[i][1] > 1:
        print('lots')
    else:

In [26]:
# here is the trick part, need to rip all the edges which are very light weight

print('Hotties len: ' + str(len(hot_names)))
print('Hotties len: ' + str(len(np.unique(np.array(hot_names)))))

Hotties len: 324
Hotties len: 324


In [124]:
hot_data['Too'].sum

<bound method Series.sum of Mama          0.0
Love          0.0
Graduation    0.0
Something     0.0
Tell          0.0
             ... 
Welcome       0.0
Hot           0.0
Song          0.0
Rockin        0.0
Spirit        0.0
Name: Too, Length: 156, dtype: float64>

In [169]:
hot_data.index

Index(['Mama', 'Love', 'Graduation', 'Something', 'Tell', 'Going', 'Day',
       'Crescendolls', 'Long', 'and',
       ...
       'Ring-A-Ling', 'Beat', 'Stronger', 'Feeling', 'Lights', 'Welcome',
       'Hot', 'Song', 'Rockin', 'Spirit'],
      dtype='object', length=156)

In [170]:
for idx in hot_data.index:
    rowsum = hot_data.loc[idx,:].sum(axis=0)
    colsum = hot_data[idx].sum(axis=0)
    print('Idx: ' + idx + '; sum of row: ' + str(rowsum))
    if rowsum < 1 and colsum < 1:
        hot_data.drop([idx], axis = 0, inplace=True)
        hot_data.drop([idx], axis = 1, inplace=True)
        print('dropping: ' + idx)

Idx: Mama; sum of row: 0.0
dropping: Mama
Idx: Love; sum of row: 0.0
dropping: Love
Idx: Graduation; sum of row: 0.0
dropping: Graduation
Idx: Something; sum of row: 0.0
Idx: Tell; sum of row: 0.0
dropping: Tell
Idx: Going; sum of row: 0.0
dropping: Going
Idx: Day; sum of row: 0.0
dropping: Day
Idx: Crescendolls; sum of row: 0.0
dropping: Crescendolls
Idx: Long; sum of row: 4.0
Idx: and; sum of row: 0.0
dropping: and
Idx: Steam; sum of row: 0.0
dropping: Steam
Idx: Dum; sum of row: 0.0
dropping: Dum
Idx: Club; sum of row: 0.0
dropping: Club
Idx: Street; sum of row: 0.0
dropping: Street
Idx: Nothing; sum of row: 0.0
dropping: Nothing
Idx: Story; sum of row: 0.0
dropping: Story
Idx: A8; sum of row: 0.0
dropping: A8
Idx: +; sum of row: 0.0
dropping: +
Idx: Pinocchio; sum of row: 0.0
dropping: Pinocchio
Idx: Momma; sum of row: 0.0
dropping: Momma
Idx: 909; sum of row: 0.0
dropping: 909
Idx: One; sum of row: 0.0
Idx: Lockdown; sum of row: 0.0
dropping: Lockdown
Idx: All; sum of row: 0.0
dro

In [131]:
hot_data.drop(['Mama'], axis = 0, inplace=True)
hot_data.drop(['Mama'], axis = 1, inplace=True)

KeyError: "['Mama'] not found in axis"

In [171]:
hot_data

,Something,Long,One,Around,Workout,Faster,Boom,Television,School,Too,...,Us,Want,World,Skit,Better,More,Rules,Harder,Stronger,Spirit
Something,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Long,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
One,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Around,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Workout,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Faster,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
Boom,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Television,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
School,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Too,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [162]:
# sum of row
rowsum = hot_data.loc['Something',:].sum(axis=0)
rowsum

0.0

In [165]:
tempdf = hot_data['Something'].sum(axis=0)
tempdf

2.0

In [185]:
gen_html(hot_mat, hot_names, 'hotties156_limit1.html')

In [151]:
not_mat, not_names = gen_chord(notlist)
# gen_html(not_mat, not_names, 'notties.html')

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
<class 'numpy.matrix'>
         Jukebox  Places  Fall  Telle   By  Customers  Naturally  Left   My  \
Jukebox      0.0     0.0   0.0    0.0  0.0        0.0        0.0   0.0  0.0   
Places       0.0     0.0   0.0    0.0  0.0        0.0        0.0   0.0  0.0   
Fall         0.0     0.0   0.0    0.0  0.0        0.0        0.0   0.0  0.0   
Telle        0.0     0.0   0.0    0.0  0.0        0.0        0.0   0.0  0.0   
By           0.0     0.0   0.0    0.0  0.0        0.0        0.0   0.0  0.0   
...          ...     ...   ...    ...  ...        ...        ...   ...  ...   
fai          0.0     0.0   0.0    0.0  0.0        0.0        0.0   0.0  0.0   
Fiesta       0.0     0.0   0.0    0.0  0.0        0.0        0.0   0.0  0.0   
les          0.0     0.0   0.0    0.0  0.0        0.0        0.0   0.0  0.0   
bones        0.0     0.0   0.0 

### STOP HERE

In [152]:
# test sklearn countvectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [159]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 1)) 
co_occurrences = bigram_vectorizer.fit_transform(unique_hotties)

In [161]:
print('Printing sparse matrix:', co_occurrences)
print('Printing dense matrix', co_occurrences.todense())
sum_occ = np.sum(co_occurrences.todense(),axis=0)
print('Sum of word-word occurrences:', sum_occ)
print('Pretty printig of co_occurrences count:')
z = zip(bigram_vectorizer.get_feature_names(),np.array(sum_occ)[0].tolist())
print(z)

Printing sparse matrix:   (0, 132)	1
  (0, 87)	1
  (0, 125)	1
  (0, 230)	1
  (1, 173)	1
  (1, 164)	1
  (1, 178)	1
  (2, 174)	1
  (2, 146)	1
  (3, 107)	1
  (3, 65)	1
  (3, 208)	1
  (4, 163)	1
  (4, 171)	1
  (4, 251)	1
  (4, 239)	1
  (5, 171)	1
  (5, 251)	1
  (5, 207)	1
  (5, 30)	1
  (5, 260)	1
  (5, 106)	1
  (6, 251)	1
  (6, 116)	1
  (7, 171)	1
  :	:
  (170, 54)	1
  (170, 284)	1
  (171, 191)	1
  (171, 68)	1
  (172, 82)	1
  (172, 34)	1
  (173, 53)	1
  (174, 179)	1
  (174, 233)	1
  (175, 47)	1
  (176, 292)	1
  (177, 1)	1
  (178, 251)	1
  (178, 55)	1
  (178, 25)	1
  (179, 302)	1
  (179, 175)	1
  (179, 140)	1
  (179, 268)	1
  (180, 101)	1
  (180, 74)	1
  (181, 6)	1
  (182, 226)	1
  (182, 112)	1
  (183, 161)	1
Printing dense matrix [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Sum of word-word occurrences: [[ 1  1  1  1  1  3  3  1  2  1  1  1  2  1  1  2  1  1  1  3  1  1  1  1
   1  1  3  1  1  1  2  1  1  1  1  4  1

In [162]:
df = pd.DataFrame(z)
print(df)

           0   1
0         36   1
1         42   1
2        837   1
3        909   1
4         a8   1
..       ...  ..
299  workout   2
300    world   3
301     yeah   1
302      you  11
303     your   5

[304 rows x 2 columns]
